# Data prepare and FC_DNN

In [1]:
# Import data_prepare modul that contains functions that we created.
from data_prepare import *
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dense, Flatten, Conv1D, MaxPooling1D, Dropout
from tensorflow.keras.optimizers import SGD, Adam
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint
from scripts.reconstruction_minimal import createAudio
import scipy.io.wavfile as wavfile

# Prepare data for learning

We prepare the data for learning neural networks. We use a function that execute the results of the last milestone get a train-ready dataset.

In [32]:
def generate_datasets(pt, feat_path=r'./features', cnn_prepare=False):
    
    r"""
    Generate train and test datasets from the one sample of the raw data.
    param pt: name of the sample
    param feat_path: path of the raw data file
  """
    
    spectrogram = np.load(os.path.join(feat_path,f'{pt}_spec.npy'))
    data = np.load(os.path.join(feat_path,f'{pt}_feat.npy'))
    
    if cnn_prepare:
        # Creating data for CNN: stacking window_size input data and connecting it to the output
        # at the end of the window
        window_size = 4
        input = np.zeros((data.shape[0] - window_size, window_size, data.shape[1]))
        output = np.zeros((spectrogram.shape[0] - window_size, spectrogram.shape[1]))
        for i in range(input.shape[0] - window_size):
          output[i, :] = spectrogram[i + window_size, :]
          input[i, :, :] = data[i:i+window_size, :]
    else:
        output = spectrogram
        input = data

    # Generate train, validation and test datasets
    X_train, X_test, Y_train, Y_test = train_test_split(input, output, test_size=0.1, shuffle=False)

    # Shuffle train data
    randomize = np.arange(len(X_train))
    np.random.shuffle(randomize)
    X_train = X_train[randomize]
    Y_train = Y_train[randomize]
    
    # Stnadardize input
    mu_input = np.mean(X_train, axis=0)
    std_input = np.std(X_train, axis=0)
    X_train = (X_train-mu_input)/std_input
    X_test = (X_test-mu_input)/std_input

    # Reduce Dimensions
    pca = PCA()
    pca.fit(X_train)
    X_train = np.dot(X_train, pca.components_[:50,:].T)
    X_test = np.dot(X_test, pca.components_[:50,:].T)

    # Standardize output
    mu_output = np.mean(Y_train, axis=0)
    std_output = np.std(Y_train, axis=0)
    Y_train = (Y_train-mu_output)/std_output
    Y_test = (Y_test-mu_output)/std_output


    return (X_train, Y_train, X_test, Y_test, mu_input, std_input, mu_output, std_output)

In [33]:
def generate_merged_dataset(cnn_prepare=False, pts_num=10):
    norm_params = {}
    X_test = []
    Y_test = []
    pts = ['sub-%02d'%i for i in range(1, pts_num+1)]
    (X_train, Y_train, x_test, y_test, mu_input, std_input, mu_output, std_output) = generate_datasets(pts[0], cnn_prepare=cnn_prepare)
    norm_params[str(0)]={'mu_input':mu_input, 'std_input':std_input, 'mu_output':mu_output, 'std_output':std_output}
    X_test.append(x_test)
    Y_test.append(y_test)
    for i, pt in enumerate(pts[1:]):
        (x_train, y_train, x_test, y_test, mu_input, std_input, mu_output, std_output) = generate_datasets(pt, cnn_prepare=cnn_prepare)
        norm_params[str(i+1)]={'mu_input':mu_input, 'std_input':std_input, 'mu_output':mu_output, 'std_output':std_output}
        X_train = np.concatenate((X_train, x_train))
        Y_train = np.concatenate((Y_train, y_train))
        X_test.append(x_test)
        Y_test.append(y_test)
    return (X_train, Y_train, X_test, Y_test, norm_params)

# Create FC-DNN network

In [29]:
def create_fndnn(input_dim, output_dim):
    model = Sequential()
    # model.add(Dense(units=150, input_dim=input_dim, activation='relu'))
    # model.add(Dropout(0.5))
    # model.add(Dense(units=100, activation='relu'))
    # # model.add(Dropout(0.5))
    # model.add(Dense(units=80, input_dim=input_dim, activation='relu'))
    # # model.add(Dropout(0.5))
    model.add(Dense(units=60, activation='relu'))
    # model.add(Dropout(0.5))
    # model.add(Dense(units=30, input_dim=input_dim, activation='relu'))
    # model.add(Dropout(0.5))
    # model.add(Dense(units=10, activation='relu'))
    # model.add(Dropout(0.5))
    model.add(Dense(units=output_dim, activation='linear'))

    optimizer=Adam(learning_rate=0.001)
    model.compile(loss='mse', optimizer=optimizer, metrics=['mae'])
    
    return model

In [5]:
def make_1d_convnet(window_size, filter_length, nb_input_series=1, nb_outputs=1, nb_filter=[40, 40]):
    #probably more filter layers should be added

    model = Sequential()
    model.add(Conv1D(filters=nb_filter[0], kernel_size=filter_length, activation='relu', input_shape=(window_size, nb_input_series)))
    model.add(Conv1D(filters=nb_filter[1], kernel_size=filter_length, activation='relu'))
    model.add(Flatten())
    model.add(Dense(nb_outputs, activation='linear'))

    optimizer=Adam(learning_rate=0.001)
    
    model.compile(loss='mse', optimizer=optimizer, metrics=['mae'])
    return model

In [36]:
def test_model(model, X_test, Y_test, norm_params):
    preds = []
    spectograms = []
    losses = []
    pts_num = len(norm_params)
    for i in range(pts_num):
        y_pred = model.predict(X_test[i])
        losses.append(mean_absolute_error(Y_test[i], y_pred))
        preds.append(y_pred * norm_params[str(i)]['std_output'] + norm_params[str(i)]['mu_output'])
        spectograms.append(Y_test[i] * norm_params[str(i)]['std_output'] + norm_params[str(i)]['mu_output'])
    mae = np.mean(losses)
    print(f'Mean absolute error of test set:{mae}')
    return preds, spectograms

In [9]:
(X_train, Y_train, X_test, Y_test, norm_params) = generate_merged_dataset(cnn_prepare=True, pts_num=1)

#Parameters for the CNN network
# one data point covers 2 * 200 ms of eeg signal
filter_length = 2 #covers 4*8*400 sec of eeg
window_size = 2 * 2 #we have two 1d max pool layers
nb_filter = [40, 40]
epochs = 1000
batch_size = 512

early_stopping=EarlyStopping(patience=50, verbose=1)
checkpointer=ModelCheckpoint(filepath='cnn.hdf5', save_best_only=True, verbose=1)
model_cnn = make_1d_convnet(window_size=window_size, filter_length=filter_length, nb_filter=nb_filter, nb_input_series=50, nb_outputs=23)
history = model_cnn.fit(X_train, Y_train, validation_split=0.1, epochs=epochs, batch_size=batch_size, verbose=2, callbacks=[checkpointer, early_stopping])

preds_cnn, spectograms = test_model(model_cnn, X_test, Y_test, norm_params)

KeyboardInterrupt: 

In [34]:
(X_train, Y_train, X_test, Y_test, norm_params) = generate_merged_dataset(pts_num=1)

In [37]:

#Parameters for the FC-DNN network
epochs = 1000
batch_size = 512

early_stopping=EarlyStopping(patience=50, verbose=1)
checkpointer=ModelCheckpoint(filepath='fcdnn.hdf5', save_best_only=True, verbose=1)
model_fcdnn = create_fndnn(input_dim=X_train.shape[1], output_dim=Y_train.shape[1])
history = model_fcdnn.fit(X_train, Y_train, validation_split=0.1, epochs=epochs, batch_size=batch_size, verbose=2, callbacks=[checkpointer, early_stopping])

preds_fcdnn, spectograms = test_model(model_fcdnn, X_test, Y_test, norm_params)

Epoch 1/1000

Epoch 1: val_loss improved from inf to 2.63956, saving model to fcdnn.hdf5
48/48 - 1s - loss: 5.3497 - mae: 1.7420 - val_loss: 2.6396 - val_mae: 1.2682 - 873ms/epoch - 18ms/step
Epoch 2/1000

Epoch 2: val_loss improved from 2.63956 to 1.63497, saving model to fcdnn.hdf5
48/48 - 0s - loss: 2.0371 - mae: 1.1079 - val_loss: 1.6350 - val_mae: 0.9920 - 190ms/epoch - 4ms/step
Epoch 3/1000

Epoch 3: val_loss improved from 1.63497 to 1.29318, saving model to fcdnn.hdf5
48/48 - 0s - loss: 1.4338 - mae: 0.9247 - val_loss: 1.2932 - val_mae: 0.8731 - 184ms/epoch - 4ms/step
Epoch 4/1000

Epoch 4: val_loss improved from 1.29318 to 1.11278, saving model to fcdnn.hdf5
48/48 - 0s - loss: 1.1802 - mae: 0.8326 - val_loss: 1.1128 - val_mae: 0.8059 - 183ms/epoch - 4ms/step
Epoch 5/1000

Epoch 5: val_loss improved from 1.11278 to 1.00294, saving model to fcdnn.hdf5
48/48 - 0s - loss: 1.0367 - mae: 0.7756 - val_loss: 1.0029 - val_mae: 0.7608 - 234ms/epoch - 5ms/step
Epoch 6/1000

Epoch 6: val_l

In [ ]:
# Viz spectrograms
def visualize_results(preds, spectograms):

    cm='viridis'
    fig, ax = plt.subplots(2, sharex=True)

    for i in range(10):
        #Plot spectrograms
        ax[0].imshow(np.flipud(spectograms[i].T), cmap=cm, interpolation=None,aspect='auto')
        ax[0].set_ylabel('Log Mel-Spec Bin')
        ax[1].imshow(np.flipud(preds[i].T), cmap=cm, interpolation=None,aspect='auto')
        ax[1].set_ylabel('Log Mel-Spec Bin')

In [41]:
# Synthesize audio files
def synthesize_audio(preds, spectograms, result_path):
    winLength = 0.05
    frameshift = 0.01
    audiosr = 16000
    pts_num = len(spectograms)
    if not os.path.isdir(result_path):
        os.makedirs(result_path)
    for i in range(pts_num):
        #Synthesize waveform from spectrogram using Griffin-Lim
        reconstructedWav = createAudio(preds[i],audiosr=audiosr,winLength=winLength,frameshift=frameshift)
        wavfile.write(os.path.join(result_path,f'{i+1}_predicted.wav'),int(audiosr),reconstructedWav)

        #For comparison synthesize the original spectrogram with Griffin-Lim
        origWav = createAudio(spectograms[i],audiosr=audiosr,winLength=winLength,frameshift=frameshift)
        wavfile.write(os.path.join(result_path,f'{i+1}_orig_synthesized.wav'),int(audiosr),origWav)

In [ ]:
visualize_results(preds=preds_fcdnn, spectograms=spectograms)
visualize_results(preds=preds_cnn, spectograms=spectograms)

In [43]:
synthesize_audio(preds=preds_fcdnn, spectograms=spectograms, result_path = 'dnn_results')

In [42]:
synthesize_audio(preds=preds_cnn, spectograms=spectograms, result_path = 'cnn_results')

NameError: name 'preds_cnn' is not defined